In [19]:
# The email scrape filters for 12/16/2023 and on


%load_ext autoreload
%autoreload 2
# import importlib
# importlib.reload()

import logging
import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from modules.scrapes import scrape
from modules.db_operations import DatabaseConnector

email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet


logging.basicConfig(filename='Email_Scraper.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)


# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass, start_date):

    logging.info('\n\nNew logging instance')

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame
    email_address = 'team@customplanet.com'
    email_pass = imap_password_customplanet
 
    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass, start_date)
    inbox = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass, start_date)
    outbox = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox.name = 'outbox'
    
    if outbox.empty == True:
        thread = None
        pass
    else: #this is a left merge on outbox
        thread = scrape.piece_together(outbox, inbox)
        thread = scrape.assign_sentiment(thread)
    
    # Specify your database connection details
    server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
    database = 'emailcampaign'
    db = 'emailcampaign'
    table_name = 'thread'

    # Instantiate the DatabaseConnector class
    db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)

    #If there are any new records, append them onto thread table
    #Update the reply_thread everytime
    if thread.empty == False:
        new_records = db_connector.append_new_records(thread)
        db_connector.update_reply_thread(thread)
        logging.info(f'Thread has been updated with {len(new_records)}')
    else:
        logging.info('Thread is none')
        print('Thread is none')
        new_records = None

    # return(inbox, outbox, thread, new_records)
    return(inbox, outbox, thread, new_records)

# The foreign constraint enforces that this must have been sent initially. 
inbox, outbox, thread, new_records  = process('Sport Special', email_address, email_pass, '11/15/2023')

#4285 emails in the inbox took 21 minutes

#Send out test emails for local baskebtall jsersyes
#Then send out for local football jerseys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You have 3 email responses to the specified search criteria Sport Special in the inbox
You have 9 email responses to the specified search criteria Sport Special in the outbox


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


No new records to send


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


Reply_Thread updated
Reply_Thread updated
Reply_Thread updated


In [21]:
thread

#How can I log out how many reply_threads have been updated?

,recipient_id,subject,from,to,date,first_message,reply,reply_thread,sentiment,sentiment_category
0,042416d0-d914-46ac-9392-442fda6d0886,Sport Special - Football Jerseys for Kearney ...,team@customplanet.com,samuel.taylor@greendot.org,2023-12-05 22:06:47+00:00,Hello Kearney Middle School]! We are excited t...,Y,"Hello, I did not have a good experience with c...",0.149810,positive
1,d7cc7cc0-1ddc-4329-ab5b-50ba823077fb,Sport Special - Football Jerseys for Plaza Mi...,team@customplanet.com,sammytaylor2006@yahoo.com,2023-12-05 22:06:45+00:00,Hello Plaza Middle (Park Hill)]! We are excite...,Y,"** NLP Testing Hello, I am very interested. Ho...",0.184982,positive
2,9cc6106c-d6c9-4a5d-af11-48af6ad1f360,Sport Special - Football Jerseys for F.L. Sch...,team@customplanet.com,2015samtaylor@gmail.com,2023-12-05 22:06:43+00:00,Hello F.L. Schlagle High School (KCKS)]! We ar...,Y,HI! How can I place an order for football jers...,0.194597,positive
3,efe3aec1-2779-4717-a01d-ba2e4ae9d399,Sport Special - Football Jerseys for Kearney ...,team@customplanet.com,samuel.taylor@greendot.org,2023-12-05 22:04:26+00:00,Hello Kearney Middle School]! We are excited t...,N,,0.000000,NaN
4,3216d0e1-75f6-468b-9e0e-609f83e98c08,Sport Special - Football Jerseys for Plaza Mi...,team@customplanet.com,sammytaylor2006@yahoo.com,2023-12-05 22:04:25+00:00,Hello Plaza Middle (Park Hill)]! We are excite...,N,,0.000000,NaN
5,74390b9a-4c47-4e38-b8ba-c1dc6d10b3ed,Sport Special - Football Jerseys for F.L. Sch...,team@customplanet.com,2015samtaylor@gmail.com,2023-12-05 22:04:23+00:00,Hello F.L. Schlagle High School (KCKS)]! We ar...,N,,0.000000,NaN
6,ce965e8a-5cb9-4f7a-b81f-78eadc65cdd9,Sport Special - Football Jerseys for Kearney ...,team@customplanet.com,campbellk@ksdr1.net,2023-12-05 22:03:58+00:00,Hello Kearney Middle School]! We are excited t...,N,,0.000000,NaN
7,0fd095b9-8121-49ae-8af0-65327c037713,Sport Special - Football Jerseys for Plaza Mi...,team@customplanet.com,davisj@parkhill.k12.mo.us,2023-12-05 22:03:56+00:00,Hello Plaza Middle (Park Hill)]! We are excite...,N,,0.000000,NaN
8,e7df2e01-7704-4f9b-bdac-7b5308af043d,Sport Special - Football Jerseys for F.L. Sch...,team@customplanet.com,Sonya.Scott@kckps.org,2023-12-05 22:03:55+00:00,Hello F.L. Schlagle High School (KCKS)]! We ar...,N,,0.000000,NaN


: 

In [13]:
logging.info('\n\nNew loggin instance')

# To DO

In [ ]:
# Change the date query of the scrapes
import datetime

# Define the date range
start_date = datetime.datetime.now() - datetime.timedelta(days=30)  # emails from the last 30 days
end_date = datetime.datetime.now()

# Convert dates to the IMAP date format
start_date_str = start_date.strftime('%d-%b-%Y')
end_date_str = end_date.strftime('%d-%b-%Y')

# Search for emails within the specified date range
search_criteria = f'(SINCE {start_date_str} BEFORE {end_date_str}) SUBJECT "{subject_line}"'
status, response = imap.search(None, search_criteria)

if status == 'OK':
    email_ids = response[0].split()
    # Process the emails found within the date range
else:
    # Handle the case where the search failed
    print("Search failed.")


In [ ]:
#Create PNGs on hoodies images using Selenium. 

#Create a seperate module for parsing the body of text for NLP
#Scrape out the body at a later date for sentiment analysis
#IF inbox is empty default to just outbox

#Look into this at some point
#Why with the reply of 'Y' does it have an empty field on the reply_thread

#Thirdly, what if the email is not directly replied to. In that case. 
# The scenarion would be the following, email campaign sent out to sammytaylor2006@yahoo.com
#The customer indirectly reaches out to CP, and not through the thread. This calls for a seperate table.
# and a seperate scrape 

#Integrate SSIS at some point for views

#Integrating prior orders and linking them in a SQL view.

#Predictive analytics based on prior orders

#Figure out why the run time is so long


#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 

#Singularize the db_operations to reference in both